In [2]:
import numpy as np
import os
from skimage.io import imread_collection

In [3]:
imgs = imread_collection('/Users/colette/Downloads/style-color-images/style/*.png')

In [5]:
len(imgs)

2184

In [ ]:
from skimage.transform import resize

imgs = [resize(x,(77,65), mode='constant', anti_aliasing=False) for x in imgs]
flatImgs = [x.flatten() for x in imgs]

df = pd.read_csv('/Users/colette/Downloads/style-color-images/style/style.csv')

In [ ]:
from sklearn.neural_network import BernoulliRBM
from sklearn.linear_model import LogisticRegression

rbm = BernoulliRBM(learning_rate=.01, n_iter=20, n_components=150)

logistic = LogisticRegression(solver='lbfgs', 
                 max_iter=10000,C=6000, multi_class='multinomial')

#Combine the two into a Pipeline
rbm_features_classifier = Pipeline(steps=[('rbm', rbm), ('logistic', logistic)])

In [ ]:
X_train = pd.read_csv('/Users/colette/Downloads/style-color-images/style/style.csv').values[:,1:]
X_train = (X_train - np.min(X_train, 0)) / (np.max(X_train, 0) + 0.0001)  # 0-1 scaling

In [ ]:
rbm_features_classifier.fit(flatImgs, Y)

In [ ]:
Y_pred = rbm_features_classifier.predict(flatImgs)

from sklearn import metrics
metrics.classification_report(Y, Y_pred)

In [ ]:
import matplotlib.pyplot as plt
from skimage.io import imshow

plt.figure(figsize=(15, 15))
for i, comp in enumerate(rbm.components_[:150]):
    plt.subplot(15, 10, i + 1)
    plt.imshow(comp.reshape((77, 65)), cmap=plt.cm.gray_r,
        interpolation='nearest')    
    plt.xticks(())
    plt.yticks(())
plt.subplots_adjust(0.08, 0.02, 0.92, 0.85, 0.08, 0.23)
plt.show()

In [ ]:
# Selected features for closer examination
toShow = [104,116,84]

plt.figure(figsize=(16, 10))
for i, comp in enumerate(toShow):
    plt.subplot(1,3,i+1)
    plt.imshow(rbm.components_[comp].reshape((77, 65)), cmap=plt.cm.gray_r,
        interpolation='nearest')
    plt.xticks(())
    plt.yticks(())
plt.show()

In [ ]:
#Create a target variable: an ID between 1 and 15 for each of the 15 subjects
Y = [[id for pose in range(1,12)] for id in range(1,16)]
#Flatten the 'list of lists' into a 2D list
Y = [num for sublist in Y for num in sublist]